# **Investigation of the bank's customer reliability**

## **1. Scenario**

The client, the bank's Credit Department, ordered a study. The goal is to determine whether the marital status and the number of children of the client affect the repayment of the credit card on time and without delay. 

## **2. Ask**

**Questions posed by the client:**

* Is there a relationship between the number of children and repayment of the loan on time?
* Is there a relationship between marital status and repayment of the loan on time?
* Is there a relationship between income level and repayment of the loan on time?
* How do the different objectives of the loan affect its repayment on time?

## **3. Prepare**

### **3.1 Data for Analysis**

I have a CSV file - clients_data.csv containing data about clients and if they were in debt. I do not have information about the quality of this data, so before answering the research questions and making a conclusion, I will need to review the data and probably clean it up.

So the study will contain the following steps:

1. Data review (Prepare)
2. Data cleaning and processing (Process)
3. Answers to research questions (Analyze)
4. General conclusion (Share)

### **3.2 Data review**

In [1]:
# import pandas and read CSV the file
import pandas as pd

data = pd.read_csv('clients_data.csv')

In [ ]:
# Take a look at the first 5 lines of our data
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose
0,1,-8437.673028,42,higher,0,married,0,F,employee,0,253875.639453,housing purchase
1,1,-4024.803754,36,secondary,1,married,0,F,employee,0,112080.014102,car purchase
2,0,-5623.422610,33,secondary,1,married,0,M,employee,0,145885.952297,housing purchase
3,3,-4124.747207,32,secondary,1,married,0,M,employee,0,267628.550329,additional education
4,0,340266.072047,53,secondary,1,civil marriage,1,F,retiree,0,158616.077870,wedding


In [3]:
# Print the basic information about the dataframe
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 21525 entries, 0 to 21524
Data columns (total 12 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   children          21525 non-null  int64  
 1   days_employed     19351 non-null  float64
 2   dob_years         21525 non-null  int64  
 3   education         21525 non-null  object 
 4   education_id      21525 non-null  int64  
 5   family_status     21525 non-null  object 
 6   family_status_id  21525 non-null  int64  
 7   gender            21525 non-null  object 
 8   income_type       21525 non-null  object 
 9   debt              21525 non-null  int64  
 10  total_income      19351 non-null  float64
 11  purpose           21525 non-null  object 
dtypes: float64(2), int64(5), object(5)
memory usage: 2.0+ MB


## **4. Process**

### **4.1 Data cleaning**

**Deleting missing values**

In [4]:
# Print the number of missing values for each column
data.isna().sum()

children               0
days_employed       2174
dob_years              0
education              0
education_id           0
family_status          0
family_status_id       0
gender                 0
income_type            0
debt                   0
total_income        2174
purpose                0
dtype: int64

There are missing values in two columns. The column with missing values, *total_income*— stores income data. The amount of income is most strongly influenced by the type of employment, so fill in the gaps in this column with the median values for each type from the income_type column.

In [5]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['total_income'].isna()), 'total_income'] = \
    data.loc[(data['income_type'] == t), 'total_income'].median()

**Handling abnormal values**

Let's process the gaps in the *days_employed* column. A negative number of days of work experience is an anomaly. Let's replace all negative values with positive ones.

In [6]:
# Take the modulus of values using abs()
data['days_employed'] = data['days_employed'].abs()

**Let's look at the unique values in the *children* column**

In [7]:
data['children'].unique()

array([ 1,  0,  3,  2, -1,  4, 20,  5])

There are two abnormal values in the children column: *(-1)* and *20*. Let's delete the lines that contain such abnormal values from the data.

In [8]:
data = data[(data['children'] != -1) & (data['children'] != 20)]

In [9]:
# Checking results
data['children'].unique()

array([1, 0, 3, 2, 4, 5])

**Fill in the missing values in the *days_employed* column with the median values for each type of employment from *income_type*.**

In [10]:
for t in data['income_type'].unique():
    data.loc[(data['income_type'] == t) & (data['days_employed'].isna()), 'days_employed'] = \
    data.loc[(data['income_type'] == t), 'days_employed'].median()

In [11]:
# Make sure that all the missing values are filled in.
data.isna().sum()

children            0
days_employed       0
dob_years           0
education           0
education_id        0
family_status       0
family_status_id    0
gender              0
income_type         0
debt                0
total_income        0
purpose             0
dtype: int64

### **4.2 Data Processing**

**For a better understanding of the data, let's look at the name of the columns and the data they contain.**

* *children* — number of children in the family
* *days_employed* — total work experience in days
* *dob_years* — client's age in years
* *education* — client's education level
* *education_id* — educational level identifier
* *family_status* — marital status
* *family_status_id* — marital status identifier
* *gender* — gender of the client
* *income_type* — type of employment
* *debt* — whether loan repayment arrears
* *total_income* — monthly income
* *purpose* — purpose of obtaining a loan

**Changing the data type in the *total_income* and *days_employed* columns to integer**

In [36]:
data['total_income'] = data['total_income'].astype(int)
data['days_employed'] = data['days_employed'].astype(int)


**Duplicate handling**

In [13]:
# Lower-case the column education
data['education'] = data['education'].str.lower()

In [14]:
# Print the number of duplicate rows
data.duplicated().sum()

np.int64(82)

In [15]:
# Delete duplicate rows
data = data.drop_duplicates()

**Data categorization**

Create a *`total_income_category`* column with categories

- 0–30000 — `'E'`;
- 30001–50000 — `'D'`;
- 50001–200000 — `'C'`;
- 200001–1000000 — `'B'`;
- 1000001 and more — `'A'`.

In [16]:
def categorize_income(income):
    try:
        if 0 <= income <= 30000:
            return 'E'
        elif 30001 <= income <= 50000:
            return 'D'
        elif 50001 <= income <= 200000:
            return 'C'
        elif 200001 <= income <= 1000000:
            return 'B'
        elif income >= 1000001:
            return 'A'
    except:
        pass

In [17]:
data['total_income_category'] = data['total_income'].apply(categorize_income)

**List of unique purposes of taking out a loan.**

In [18]:
data['purpose'].unique()

array(['housing purchase', 'car purchase', 'additional education',
       'wedding', 'real estate transactions', 'education', 'for wedding',
       'buying housing for family', 'real estate purchase',
       'buying commercial real estate', 'buying residential real estate',
       'construction of own property', 'real estate',
       'real estate construction', 'buying a used car', 'buying own car',
       'commercial real estate transactions',
       'residential real estate construction', 'housing',
       'transactions with own real estate', 'cars', 'pursue education',
       'used car transaction', 'getting education', 'car',
       'getting additional education', 'buying own housing',
       'getting higher education', 'own car', 'car transaction',
       'professional education', 'higher education',
       'buying housing for renting', 'for car purchase',
       'housing renovation', 'pursue higher education'], dtype=object)

Create a new purpose_category column, which will include the following categories:

- car
- real estate
- wedding
- education

In [29]:
def categorize_purpose(row):
    try:
        if 'car' in row:
            return 'car'
        elif 'estate' in row or 'hous' in row:
            return 'real estate'
        elif 'wedding' in row:
            return 'wedding'
        elif 'education' in row:
            return 'education'
    except:
        return 'other'

In [30]:
data['purpose_category'] = data['purpose'].apply(categorize_purpose)

In [31]:
data['purpose_category'].unique()

array(['real estate', 'car', 'education', 'wedding', None], dtype=object)

**Let's look at the summary statistics for each column to check them for outliers**

In [38]:
data.describe()

,children,days_employed,dob_years,education_id,family_status_id,debt,total_income
count,21320.000000,21320.000000,21320.000000,21320.000000,21320.000000,21320.000000,2.132000e+04
mean,0.474109,67045.826829,43.270638,0.816932,0.974625,0.081238,1.653635e+05
std,0.752428,139195.093976,12.573537,0.549381,1.422107,0.273207,9.833479e+04
min,0.000000,24.000000,0.000000,0.000000,0.000000,0.000000,2.066700e+04
25%,0.000000,1024.000000,33.000000,1.000000,0.000000,0.000000,1.074905e+05
50%,0.000000,1996.000000,42.000000,1.000000,0.000000,0.000000,1.425940e+05
75%,1.000000,5320.250000,53.000000,1.000000,1.000000,0.000000,1.958730e+05
max,5.000000,401755.000000,75.000000,4.000000,4.000000,1.000000,2.265604e+06


**Conclusion:**

* The *children* column:

    There are no errors. The minimum number of children is 0. The maximum is 5.

* The *days_employed* column:

    There are no negative values. The minimum value is 24 days. The maximum value is 401,755 years of service. One could assume that the value is mistakenly specified in hours, but the values of the first and third quartiles look plausible, so I assume that the maximum value is outlier.

* The *dob_years* column:

    The minimum age value in years is 0 - Outlier

* The *total_income* column:

    No deviations

**Column *dob_years***

In [39]:
# Let's look at the number of rows in the dataframe,
# where the age is less than 18 years (minimum age of the client)
data.loc[data['dob_years'] < 18, 'dob_years'].count()

np.int64(100)

In [42]:
# Let's see if the rows with an age of less than 18 are zero. To do this,
# let's count the number of rows less than 18 years old and equal to 0.
data.loc[(data['dob_years'] < 18) & (data['dob_years'] == 0), 'dob_years'].count()

np.int64(100)

Conclusion: 100 lines will contain the age of the borrower equal to 0, which is an error in the data. The error could probably have occurred due to filling in the missing values with zeros. We'll treat them as omissions. Let's replace the zero values with the average values that we calculate for each type of employment(*income_type*).

In [44]:
# Let's go through the for loop through the unique values of the employment type column,
# find the average value, and fill in the gaps with the values obtained.
for t in data['income_type'].unique():
    mean_value = data.loc[data['income_type'] == t, 'dob_years'].mean()
    data.loc[(data['dob_years'] == 0) & (data['income_type'] == t), 'dob_years'] = mean_value

In [45]:
data.loc[data['dob_years'] == 0, 'dob_years'].count()

np.int64(0)

**Column *education***

In [ ]:
# Looking at the values of the education column
data['education'].sort_values().value_counts()

education
secondary            15065
higher                5226
unfinished higher      741
primary                282
academic degree          6
Name: count, dtype: int64

The data is fine.

## **5. Analysis**

### **5.1 Is there a relationship between the number of children and repayment of the loan on time?**

To answer this question, I will do the following:
1. Create an additional column in the dataframe, in which I will categorize clients by the number of children.
2. Create a pivot table.

Let's create a function that will display the category of people by the number of children:
* *no children*, if there are no children
* *have children*, if 1 or 2 children 
* *have many children*, if 3 or more children

In [52]:
def children_category(children):
    if 1 <= children <= 2:
        return 'have children'
    if children >= 3:
        return 'have many children'
    return 'no children'

In [53]:
# Create a new column 'parenthood' and fill it with the results of the function
data['parenthood'] = data['children'].apply(children_category)

In [63]:
# Check the current state of the dataframe
data.head()

,children,days_employed,dob_years,education,education_id,family_status,family_status_id,gender,income_type,debt,total_income,purpose,total_income_category,purpose_category,parenthood
0,1,8437,42.0,higher,0,married,0,F,employee,0,253875,housing purchase,B,real estate,have children
1,1,4024,36.0,secondary,1,married,0,F,employee,0,112080,car purchase,C,car,have children
2,0,5623,33.0,secondary,1,married,0,M,employee,0,145885,housing purchase,C,real estate,no children
3,3,4124,32.0,secondary,1,married,0,M,employee,0,267628,additional education,B,education,have many children
4,0,340266,53.0,secondary,1,civil marriage,1,F,retiree,0,158616,wedding,C,wedding,no children


**Create a pivot table**

In [55]:
# Creating pivot_table()
debt_children = data.pivot_table(index='parenthood', columns='debt', values='gender', aggfunc='count')
# Creating new columns
debt_children.columns = ['no_debt', 'debt']
# Calculating the number of debtors
debt_children['number_of_debtors'] = debt_children['debt'] / (debt_children['debt'] + debt_children['no_debt'])
# Converting the values in the number_of_debtors column to percentages
debt_children['number_of_debtors'] = debt_children['number_of_debtors'].map('{:.1%}'.format)
# Sort by column number_of_debtors in descending order
debt_children.sort_values(by='number_of_debtors', ascending=False)

,no_debt,debt,number_of_debtors
parenthood,,,
have children,6220,638,9.3%
have many children,349,31,8.2%
no children,13019,1063,7.5%


**Conclusion: Having children increases the likelihood of debt.** this is probably due to additional expenses for children. However, it can be noted that in *"have many children"* category the level of debtors is lower (8.2%) than in families with one or two children (9.3%). However, the number of borrowers in the *"have many children"* category may not be enough for this comparison. The lowest rate of debtors is for the category of citizens without children (7.5%).

### **5.2 Is there a relationship between marital status and repayment of the loan on time?**

**Create a pivot table**

Let's create a pivot table as in the answer to the previous question above

In [59]:
debt_family_status = data.pivot_table(index='family_status', columns='debt', values='gender', aggfunc='count')
debt_family_status.columns = ['no_debt', 'debt']
debt_family_status['number_of_debtors'] = debt_family_status['debt'] / (debt_family_status['debt'] + debt_family_status['no_debt'])
debt_family_status['number_of_debtors'] = debt_family_status['number_of_debtors'].map('{:.1%}'.format)
debt_family_status.sort_values(by='number_of_debtors', ascending=False)

,no_debt,debt,number_of_debtors
family_status,,,
not married,2523,273,9.8%
civil marriage,3741,385,9.3%
married,11331,927,7.6%
divorced,1105,84,7.1%
widow/widower,888,63,6.6%


**Output: Unmarried people and those in civil partnerships are more at risk of becoming debtors** Other categories that have been married or are currently married have lower percentages of debtors. The widower/widow category has the lowest percentage (6.6%). 

### **5.3  Is there a relationship between income level and repayment of the loan on time?**

Let me remind you of the income categorization that I created earlier:

- 0–30000 — 'E';
- 30001–50000 — 'D';
- 50001–200000 — 'C';
- 200001–1000000 — 'B';
- 1000001 and more — 'A'.

**Create a pivot table**

In [60]:
debt_total_income = data.pivot_table(index='total_income_category', columns='debt', values='gender', aggfunc='count')
debt_total_income.columns = ['no_debt', 'debt']
debt_total_income['number_of_debtors'] = debt_total_income['debt'] / (debt_total_income['debt'] + debt_total_income['no_debt'])
debt_total_income['number_of_debtors'] = debt_total_income['number_of_debtors'].map('{:.1%}'.format)
debt_total_income.sort_values(by='number_of_debtors', ascending=False)

,no_debt,debt,number_of_debtors
total_income_category,,,
E,20,2,9.1%
C,14557,1353,8.5%
A,23,2,8.0%
B,4660,354,7.1%
D,328,21,6.0%


**Conclusion: there is a direct relationship between the number of people in debt and their income level.** The more a person earns, the less likely they are to have financial difficulties.

### **5.4 How do the different objectives of the loan affect its repayment on time?**

**Create a pivot table**

In [61]:
debt_purpose_category = data.pivot_table(index='purpose_category', columns='debt', values='gender', aggfunc='count')
debt_purpose_category.columns = ['no_debt', 'debt']
debt_purpose_category['share_of_debtors'] = debt_purpose_category['debt'] / (debt_purpose_category['debt'] + debt_purpose_category['no_debt'])
debt_purpose_category['share_of_debtors'] = debt_purpose_category['share_of_debtors'].map('{:.1%}'.format)
debt_purpose_category.sort_values(by='share_of_debtors', ascending=False)

,no_debt,debt,share_of_debtors
purpose_category,,,
car,3879,400,9.3%
education,3619,369,9.3%
wedding,2122,183,7.9%
real estate,9381,739,7.3%


**Conclusion: Citizens who take out loans for education and car operations are more at risk of becoming debtors.** These two categories have the same percentage of debtors (9.3%). Citizens who take out a loan for real estate transactions are more responsible for its repayment.

## **6. General conclusion**

* Having children increases the likelihood of debt
* Unmarried people and those in civil partnerships are more at risk of becoming debtors
* The less a person earns, the more likely they are to incur debt
* Citizens who take out loans for education and car operations are more at risk of becoming debtors